In [1]:
from googleapiclient.discovery import build
from datetime import datetime, timedelta
import pandas as pd

In [ ]:
# global variables
MAX_RESULTS = 30

**API Key: Lorenzo**

In [6]:
# open the text file containing the API key
with open("../authentication/YouTube_Data_API_Key.txt", "r") as file:
    API_KEY = file.read().strip() 

**API Key: Ishwarya**

In [ ]:
# open the text file containing the API key
with open("YOUR PATH TO THE API KEY HERE", "r") as file:
    API_KEY = file.read().strip() 

In [8]:
# function to search for videos given a specific query. Requires max_results, published_after, and published_before both in datetime format
def search_videos(query, max_results=MAX_RESULTS, published_after=None, published_before=None):
    youtube = build('youtube', 'v3', developerKey=API_KEY)

    # convert datetime objects to ISO 8601 string format
    published_after_string = published_after.strftime('%Y-%m-%dT%H:%M:%SZ') if published_after else None
    published_before_string = published_before.strftime('%Y-%m-%dT%H:%M:%SZ') if published_before else None

    ## Two requests are created, to separately search for medium and long lenght videos.
    ## This ensures that we don't collect and YouTube Shorts videos, which we are not interested in.

    # construct request for medium lenght videos
    search_request_medium = youtube.search().list(
        q=query, 
        part="snippet", 
        type="video", 
        maxResults=max_results, 
        order="viewCount",
        videoDuration="medium", 
        publishedAfter=published_after_string, 
        publishedBefore=published_before_string)
    
    # construct request for long videos
    search_request_long = youtube.search().list(
        q=query, 
        part="snippet", 
        type="video", 
        maxResults=max_results, 
        order="viewCount",
        videoDuration="long", 
        publishedAfter=published_after_string, 
        publishedBefore=published_before_string)
    
    # execute both search requests and store the response
    search_response_medium = search_request_medium.execute()
    search_response_long = search_request_long.execute()

    # save video IDs into lists
    video_ids_medium = [item['id']['videoId'] for item in search_response_medium['items']]
    video_ids_long = [item['id']['videoId'] for item in search_response_long['items']]

    # construct request to retrieve data about found videos
    video_request_medium = youtube.videos().list(part="snippet,statistics", id=",".join(video_ids_medium))
    video_request_long = youtube.videos().list(part="snippet,statistics", id=",".join(video_ids_long))

    # execute both video requests and store the response
    video_response_medium = video_request_medium.execute()
    video_response_long = video_request_long.execute()

    # create empty list videos_data, in which each list item is a dictionary of metadata about one video
    videos_data = []
    # loop through the medium length videos and append to videos_data
    for item in video_response_medium['items']:
        videos_data.append({
            'title': item['snippet']['title'],
            'publish_date': item['snippet']['publishedAt'],
            'description': item['snippet']['description'],
            'video_id': item['id'],
            'view_count': int(item['statistics']['viewCount']) if 'viewCount' in item['statistics'] else 0
        })
    # loop through the long videos and append to videos_data
    for item in video_response_long['items']:
        videos_data.append({
            'title': item['snippet']['title'],
            'publish_date': item['snippet']['publishedAt'],
            'description': item['snippet']['description'],
            'video_id': item['id'],
            'view_count': int(item['statistics']['viewCount']) if 'viewCount' in item['statistics'] else 0
        })
    # convert the list of dictionaries to a dataframe and return
    return pd.DataFrame(videos_data)

In [20]:
# specify start and end dates
start_date = datetime(2017, 1, 1)
end_date = datetime(2017, 12, 31)
# call the search videos() function
videos_df = search_videos("artificial intelligence", MAX_RESULTS, start_date, end_date)

In [ ]:
# inspect the data
videos_df.head(10)

In [27]:
def get_video_comments(api_key, video_id, max_results=20):
    youtube = build('youtube', 'v3', developerKey=API_KEY)

    # Call the API to retrieve comments
    response = youtube.commentThreads().list(
        part='snippet',
        videoId=video_id,
        maxResults=max_results
    ).execute()

    comments = []
    for item in response['items']:
        comment = item['snippet']['topLevelComment']['snippet']['textDisplay']
        comments.append(comment)

    return comments

In [28]:
comments = get_video_comments(API_KEY, "Bg_tJvCA8zw") # TEST: get 20 comments for one specific video

# loop through the comments and print them
for i, comment in enumerate(comments, start=1):
    print(f"comment {i}: {comment}")

comment 1: I knew Jennifer Lawrence wasn&#39;t a real person
comment 2: Robot + feeling = love
comment 3: Beautiful
comment 4: Tu chizz badi hain mast 770 k dollers right
comment 5: Bluetooth
comment 6: DJs ❤
comment 7: she is the one that had remarks about the human race and if it would make it, she said ,no.
comment 8: <a href="https://www.youtube.com/watch?v=Bg_tJvCA8zw&amp;t=304">5:04</a> yep we’re doomed 💀
comment 9: 😮
comment 10: The interaction with Sophia is creepy and unsettling but I don’t think the robot noticed.
comment 11: Jimmy Fallon always looks, talks and gesters like a perv.
comment 12: Jimmy is so lovely 🤩
comment 13: Pourquoi n&#39;a-t-elle pas de cheveux ?
comment 14: Avengers: Age of Sophia
comment 15: “It’s my great start to dominate human race” jit ain’t jokin’
comment 16: Without a natural female human - society will never continue except via robot technology! That is truthful not fabricated deceit. God the true heavenly Good God our Heavenly Alnighty Lord God 